In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn import model_selection, preprocessing, metrics
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from utils import classification_util as cu
from nn_framework import *

## Load Data

In [3]:
data_path = r'../Ex1/data/Loan/loan-10k.lrn.csv'
df = pd.read_csv(data_path)
df

,ID,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,home_ownership,annual_inc,...,debt_settlement_flag,issue_d_month,issue_d_year,earliest_cr_line_month,earliest_cr_line_year,last_pymnt_d_month,last_pymnt_d_year,last_credit_pull_d_month,last_credit_pull_d_year,grade
0,24341,12500.0,12500.0,12500.0,36 months,7.21,387.17,< 1 year,MORTGAGE,81000.0,...,N,6,2018,6,2000,2,2019,2,2019,A
1,67534,33850.0,33850.0,33775.0,60 months,20.99,915.57,1 year,MORTGAGE,80000.0,...,N,10,2015,9,1984,2,2019,2,2019,E
2,35080,10000.0,10000.0,10000.0,60 months,20.00,264.94,< 1 year,RENT,36580.0,...,N,9,2017,10,2006,1,2018,11,2018,D
3,4828,20250.0,20250.0,20250.0,36 months,14.31,695.15,9 years,RENT,48700.0,...,N,0,2015,6,1996,6,2016,9,2017,C
4,59259,25000.0,25000.0,25000.0,36 months,14.99,866.52,1 year,MORTGAGE,85000.0,...,N,11,2016,0,2002,2,2019,2,2019,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,6644,20000.0,20000.0,20000.0,60 months,16.02,486.58,5 years,OWN,54000.0,...,N,8,2017,11,1991,2,2019,2,2019,C
9996,25910,7500.0,7500.0,7500.0,36 months,9.49,240.22,10+ years,RENT,46386.0,...,N,10,2014,4,2004,3,2017,6,2018,B
9997,95698,20750.0,20750.0,20750.0,60 months,15.05,494.19,< 1 year,RENT,185000.0,...,N,4,2017,2,2004,2,2019,2,2019,C
9998,27371,11000.0,11000.0,11000.0,60 months,16.29,269.20,1 year,RENT,38500.0,...,N,9,2014,8,2006,3,2016,10,2016,D


## Experiments

In [4]:
def transformer(X):
        log_tranform_col = ['int_rate', 'annual_inc', 'total_rev_hi_lim', 'tot_hi_cred_lim', 'total_bc_limit']
        c_root_tranform_col = ['installment', 'total_acc', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 
                            'total_il_high_credit_limit', 'total_bal_ex_mort', 'avg_cur_bal', 'bc_open_to_buy', 
                            'revol_bal', 'total_rec_int', 'last_pymnt_amnt', 'tot_coll_amt','tot_cur_bal']

        for col in c_root_tranform_col:
            X[col] = np.cbrt(X[col])
        for col in log_tranform_col:
            X[col] = np.log(X[col])
        return X

In [5]:
def train_test(X, y):
    X_tr, X_te, y_tr, y_te = train_test_split(X,y, random_state=40, stratify=y, test_size=0.2)
    X_te_id = X_te.pop('ID')
    X_tr_id = X_tr.pop('ID')
    return X_tr, X_te, y_tr, y_te, X_te_id, X_tr_id

In [6]:
nn = NNFramework()
data = df.copy()

In [7]:
y = data.pop('grade')
# X = data.pop('addr_state')
X = data
X = transformer(X)
cols = list(X.select_dtypes('object').columns)
nn.fit_encoder(X, cols)
X = nn.encode_dataset(X)
X_tr, X_te, y_tr, y_te, X_te_id, X_tr_id = train_test(X,y)

In [8]:
random_seed = 1038
scaling = True
oversampling = True

scaler = preprocessing.StandardScaler() if scaling else None

In [9]:
activation_functions = ['relu', 'sigmoid']
learning_rates = [0.0001, 0.001, 0.01, 0.1, 0.5]
hidden_layer_sizes = [(5,), (32,), (16, 16), (10, 5, 5), (16, 8, 8), (64, 32, 32),]

# activation_functions = ['relu']
# learning_rates = [0.0001]
# hidden_layer_sizes = [(5,), (32,), (16, 16),]

In [10]:
methods = []

for af in activation_functions:
    for lr in learning_rates:
        for hls in hidden_layer_sizes:
            methods.append((f'MLP-{af}-{lr}-{hls}', MLP(n_iter=1000, activation_function=af, learning_rate=lr, hidden_layer_sizes=hls)))
    
pipelines = cu.define_pipelines(methods, scaler=scaler, oversampling=oversampling)



In [11]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
cv_num = 5
model_params = {}
models = {}
models_lists = {}

for model_name, pipeline in pipelines.items():
    cv_results = cross_validate(pipeline, X, y, cv=cv_num, scoring='f1_macro', return_estimator=True, n_jobs=10)

    models[model_name] = cv_results['estimator']
    model_params[model_name] = {}
    models_lists[model_name] = {}

    num_cols = ['test_score', 'fit_time', 'score_time']

    for num_col in num_cols:
        models_lists[model_name][num_col] = cv_results[num_col]
        model_params[model_name][f'{num_col}_mean'] = cv_results[num_col].mean()
        model_params[model_name][f'{num_col}_std'] = cv_results[num_col].std()
    
    model_params[model_name]['parameter_num'] = cv_results['estimator'][0][model_name].number_of_params_
    model_params[model_name]['hidden_layer_sizes'] = cv_results['estimator'][0][model_name].hidden_layer_sizes
    model_params[model_name]['activation_function'] = cv_results['estimator'][0][model_name].activation_function
    model_params[model_name]['learning_rate'] = cv_results['estimator'][0][model_name].learning_rate
    models_lists[model_name]['converged'] = [e[model_name].converged_ for e in cv_results['estimator']]
    models_lists[model_name]['validation_losses'] = [e[model_name].validation_losses_ for e in cv_results['estimator']]
    models_lists[model_name]['training_losses'] = [e[model_name].training_losses_ for e in cv_results['estimator']]
    model_params[model_name]['num_iter'] = np.array(list([len(e[model_name].training_losses_) for e in cv_results['estimator']])).mean()

    print(model_name)
    print(
        f"f1 scores: {models_lists[model_name]['test_score']}\n" +
        f"f1 mean: {model_params[model_name]['test_score_mean']:.3f}\n" +
        f"f1 std: {model_params[model_name]['test_score_std']:.3f}\n"
    )
    print('----------------------------------------------------------------------------------------------------')
    

Loss did not go down for 10 iterations. Stopping training at iteration 215...
Loss did not go down for 10 iterations. Stopping training at iteration 212...
Loss did not go down for 10 iterations. Stopping training at iteration 216...
Loss did not go down for 10 iterations. Stopping training at iteration 222...
Loss did not go down for 10 iterations. Stopping training at iteration 215...
MLP-relu-0.0001-(5,)
f1 scores: [0.67932916 0.6675278  0.67802017 0.69476442 0.71516773]
f1 mean: 0.687
f1 std: 0.017

----------------------------------------------------------------------------------------------------
Loss did not go down for 10 iterations. Stopping training at iteration 177...
Loss did not go down for 10 iterations. Stopping training at iteration 190...
Loss did not go down for 10 iterations. Stopping training at iteration 193...
Loss did not go down for 10 iterations. Stopping training at iteration 194...
Loss did not go down for 10 iterations. Stopping training at iteration 193...


/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:189: RuntimeWarning: divide by zero encountered in log
  loss -= (idx == y)*np.log(y_prob)
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:189: RuntimeWarning: invalid value encountered in multiply
  loss -= (idx == y)*np.log(y_prob)


Loss did not go down for 10 iterations. Stopping training at iteration 72...
Loss did not go down for 10 iterations. Stopping training at iteration 85...
Loss did not go down for 10 iterations. Stopping training at iteration 81...
Loss did not go down for 10 iterations. Stopping training at iteration 87...


/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:189: RuntimeWarning: divide by zero encountered in log
  loss -= (idx == y)*np.log(y_prob)
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:189: RuntimeWarning: invalid value encountered in multiply
  loss -= (idx == y)*np.log(y_prob)
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: overflow encountered in exp
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])
/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:108: RuntimeWarning: invalid value encountered in divide
  return (np.exp(X)/np.sum(np.exp(X), axis=1)[:, None])


As a punishment we return a model with randomly initialized weights.
MLP-relu-0.0001-(10, 5, 5)
f1 scores: [0.71559736 0.66564361 0.09680415 0.69734959 0.7173087 ]
f1 mean: 0.579
f1 std: 0.242

----------------------------------------------------------------------------------------------------
Loss did not go down for 10 iterations. Stopping training at iteration 81...


...

Loss did not go down for 10 iterations. Stopping training at iteration 18...


/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:85: RuntimeWarning: overflow encountered in exp
  return 1 / (1+np.exp(-X))


Loss did not go down for 10 iterations. Stopping training at iteration 27...
MLP-sigmoid-0.5-(16, 16)
f1 scores: [0.0639725  0.08075986 0.06111319 0.11670335 0.1720944 ]
f1 mean: 0.099
f1 std: 0.042

----------------------------------------------------------------------------------------------------


/home/lexi/ML/ml_ss23_group13/Ex2/MLP.py:85: RuntimeWarning: overflow encountered in exp
  return 1 / (1+np.exp(-X))


Loss did not go down for 10 iterations. Stopping training at iteration 10...
Loss did not go down for 10 iterations. Stopping training at iteration 10...
Loss did not go down for 10 iterations. Stopping training at iteration 10...
Loss did not go down for 10 iterations. Stopping training at iteration 10...
Loss did not go down for 10 iterations. Stopping training at iteration 10...
MLP-sigmoid-0.5-(10, 5, 5)
f1 scores: [0.0639725  0.06388642 0.06388642 0.06388642 0.06388642]
f1 mean: 0.064
f1 std: 0.000

----------------------------------------------------------------------------------------------------
Loss did not go down for 10 iterations. Stopping training at iteration 10...
Loss did not go down for 10 iterations. Stopping training at iteration 10...
Loss did not go down for 10 iterations. Stopping training at iteration 10...
Loss did not go down for 10 iterations. Stopping training at iteration 10...
Loss did not go down for 10 iterations. Stopping training at iteration 10...
MLP-

In [19]:
df_param = pd.DataFrame(model_params).transpose()
df_param = df_param.reset_index(drop=False)
df_param = df_param.rename(columns={'index': 'model'})

In [20]:
df_param

,model,test_score_mean,test_score_std,fit_time_mean,fit_time_std,score_time_mean,score_time_std,parameter_num,hidden_layer_sizes,activation_function,learning_rate,num_iter
0,"MLP-relu-0.0001-(5,)",0.686962,0.016567,195.440456,10.348554,0.012875,0.000351,912,"(5,)",relu,0.0001,217.0
1,"MLP-relu-0.0001-(32,)",0.637483,0.015374,227.997793,8.737823,0.018614,0.002242,5799,"(32,)",relu,0.0001,190.4
2,"MLP-relu-0.0001-(16, 16)",0.673423,0.015513,220.494422,16.713127,0.016936,0.005812,3175,"(16, 16)",relu,0.0001,140.0
3,"MLP-relu-0.0001-(10, 5, 5)",0.578541,0.241584,177.741284,10.017526,0.022197,0.010148,1867,"(10, 5, 5)",relu,0.0001,85.4
4,"MLP-relu-0.0001-(16, 8, 8)",0.539063,0.229464,137.542779,9.866459,0.012556,0.000707,3055,"(16, 8, 8)",relu,0.0001,90.2
5,"MLP-relu-0.0001-(64, 32, 32)",0.045744,0.00276,0.970717,0.513438,0.016173,0.001392,14503,"(64, 32, 32)",relu,0.0001,0.0
6,"MLP-relu-0.001-(5,)",0.694258,0.028037,74.046086,4.702816,0.012439,0.000797,912,"(5,)",relu,0.001,87.0
7,"MLP-relu-0.001-(32,)",0.62958,0.006847,47.40502,3.317254,0.015678,0.003852,5799,"(32,)",relu,0.001,43.0
8,"MLP-relu-0.001-(16, 16)",0.073786,0.002305,2.5712,0.444815,0.015335,0.003307,3175,"(16, 16)",relu,0.001,1.2
9,"MLP-relu-0.001-(10, 5, 5)",0.089531,0.003964,1.779617,0.256817,0.014233,0.001285,1867,"(10, 5, 5)",relu,0.001,0.8


In [21]:
df_param.to_csv(r'results/loan_params.csv', index=False)

In [12]:
df_param = pd.read_csv(r'results/loan_params.csv')

In [13]:
df_param

,model,test_score_mean,test_score_std,fit_time_mean,fit_time_std,score_time_mean,score_time_std,parameter_num,hidden_layer_sizes,activation_function,learning_rate,num_iter
0,"MLP-relu-0.0001-(5,)",0.686962,0.016567,195.440456,10.348554,0.012875,0.000351,912,"(5,)",relu,0.0001,217.0
1,"MLP-relu-0.0001-(32,)",0.637483,0.015374,227.997793,8.737823,0.018614,0.002242,5799,"(32,)",relu,0.0001,190.4
2,"MLP-relu-0.0001-(16, 16)",0.673423,0.015513,220.494422,16.713127,0.016936,0.005812,3175,"(16, 16)",relu,0.0001,140.0
3,"MLP-relu-0.0001-(10, 5, 5)",0.578541,0.241584,177.741284,10.017526,0.022197,0.010148,1867,"(10, 5, 5)",relu,0.0001,85.4
4,"MLP-relu-0.0001-(16, 8, 8)",0.539063,0.229464,137.542779,9.866459,0.012556,0.000707,3055,"(16, 8, 8)",relu,0.0001,90.2
5,"MLP-relu-0.0001-(64, 32, 32)",0.045744,0.002760,0.970717,0.513438,0.016173,0.001392,14503,"(64, 32, 32)",relu,0.0001,0.0
6,"MLP-relu-0.001-(5,)",0.694258,0.028037,74.046086,4.702816,0.012439,0.000797,912,"(5,)",relu,0.0010,87.0
7,"MLP-relu-0.001-(32,)",0.629580,0.006847,47.405020,3.317254,0.015678,0.003852,5799,"(32,)",relu,0.0010,43.0
8,"MLP-relu-0.001-(16, 16)",0.073786,0.002305,2.571200,0.444815,0.015335,0.003307,3175,"(16, 16)",relu,0.0010,1.2
9,"MLP-relu-0.001-(10, 5, 5)",0.089531,0.003964,1.779617,0.256817,0.014233,0.001285,1867,"(10, 5, 5)",relu,0.0010,0.8


In [14]:
df_param.sort_values(['test_score_mean'], ascending=False).head(5)

,model,test_score_mean,test_score_std,fit_time_mean,fit_time_std,score_time_mean,score_time_std,parameter_num,hidden_layer_sizes,activation_function,learning_rate,num_iter
35,"MLP-sigmoid-0.0001-(64, 32, 32)",0.742338,0.021881,1236.012081,20.893088,0.019591,0.003391,14503,"(64, 32, 32)",sigmoid,0.0001,449.0
41,"MLP-sigmoid-0.001-(64, 32, 32)",0.741701,0.022078,229.434758,7.056314,0.023255,0.006144,14503,"(64, 32, 32)",sigmoid,0.0010,81.2
47,"MLP-sigmoid-0.01-(64, 32, 32)",0.715269,0.012124,57.154692,8.047412,0.019277,0.001969,14503,"(64, 32, 32)",sigmoid,0.0100,20.8
6,"MLP-relu-0.001-(5,)",0.694258,0.028037,74.046086,4.702816,0.012439,0.000797,912,"(5,)",relu,0.0010,87.0
40,"MLP-sigmoid-0.001-(16, 8, 8)",0.692447,0.018932,242.074241,10.673677,0.017482,0.002108,3055,"(16, 8, 8)",sigmoid,0.0010,119.8


In [18]:
modelsss = df_param.sort_values(['test_score_mean'], ascending=False).head(5)['model']
print(X_tr.shape)
results_df2 = pd.DataFrame()
for mod in modelsss[0:1]:
    print(mod)
    pipelines[mod].fit(X_tr, y_tr)
    y_pred = pipelines[mod].predict(X_te)
    eval_results = cu.eval(y_pred, y_te)
    results_df2 = results_df2.append({
        'Model': mod,
        'cm': eval_results['cm'].flatten(),
        'acc': eval_results['acc'],
        'balanced_acc': eval_results['balanced_acc'],
        'precision': eval_results['precision'],
        'recall': eval_results['recall'],
        'f-score': eval_results['f-score'],
        'support': eval_results['support']
    }, ignore_index=True)

(8000, 172)
MLP-sigmoid-0.0001-(64, 32, 32)
Loss did not go down for 10 iterations. Stopping training at iteration 431...
Evaluation metrics:


array([[0.9532967 , 0.0467033 , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.078125  , 0.86458333, 0.05729167, 0.        , 0.        ,
        0.        , 0.        ],
       [0.00167224, 0.04347826, 0.89464883, 0.06020067, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.05172414, 0.84827586, 0.1       ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.2016129 , 0.68548387,
        0.11290323, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.43243243,
        0.45945946, 0.10810811],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.72727273, 0.27272727]])

acc: 0.8655
balanced_acc: 0.7112107616121307

Macro-averaged precision: 0.7201077717644317
Macro-averaged recall: 0.7112107616121307
Macro-averaged f-score: 0.7126537754001301
Macro-averaged support: None


In [19]:
results_df2.to_csv("results/loan_trainset_big_model.csv", index=False)

In [20]:
results_df2

,Model,cm,acc,balanced_acc,precision,recall,f-score,support
0,"MLP-sigmoid-0.0001-(64, 32, 32)","[0.9532967032967034, 0.046703296703296704, 0.0...",0.8655,0.711211,0.720108,0.711211,0.712654,None
